### 1. Definição do problema
> O objetivo deste trabalho é apresentar uma análise sobre os preços de produtos importados pelo Brasil, comparando os valores declarados numa operação de importação com os preços praticados no comércio internacional.
>Para isso, primeiramente, será realizado um levantamento de preços junto ao mercado mundial para um grupo de mercadorias de alto valor agregado.
> Em seguida, buscar-se-ão os preços desses mesmos produtos declarados em processos de importação de mercadorias.
> Por fim, far-se-á a comparação entre as médias desses valores, visando a uma conclusão quanto a indícios de fraudes - subfaturamento ou superfaturamento - nas operações de importação realizadas pelo importador nacional.

### 2. A coleta de dados
> Os datasets usados neste trabalho foram coletados de fontes diversas. 
> Definiu-se que os sites __www.aliexpress.com__ e __www.alibaba.com__, notórios fornecedores de produtos diversos, serão objeto de webscrapping realizado com a ferramenta de código aberto _Beautifull Soup_ e _Selenium_, as quais serão utilizadas para coletar uma gama de aproximadamente 1 mil registros de cada um desses sites.
> Para delimitar o escopo da análise, será definido apenas uma gama de produtos, que apresenta uma variação grande de preços: __display lcd para aparelhos celulares__.

__Bold por favor__ <br>
_Italico Olha isso_ <br>
$ a + bX$ <br>
__________ dfjlkaj dlkjf lkajkdj
> item 
>>item2
>>>item3
- item 1
- item 2
1. item 1
2. item 2 <br>
<font color=blue>"font color=blue"</font><br>
paragrafog <br> outro paragrafo<br>
#isto é comentário

In [21]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

#bibliotecas para requisição de páginas dinâmicas
from selenium import webdriver 
#from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
import time
import os
import sys
import re
    
def alteraIdioma(driver):
    driver.find_element_by_xpath('//*[@id="switcher-info"]').click()#/html/body/div[2]/div/div/div[2]/div[2]/div/div[1]/div[2]/#div[1]/span[2]/span/span[2]/span
    time.sleep(3)
    painel = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div[4]/div/div')
    painel.click()
    #painel country selector switcher
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div[4]/div/div/div/div[1]/div/a[1]/span/span').click()
    time.sleep(3)
    painel.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[4]/div/div/div/div[1]/div/div[1]/ul/li[224]").click()
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div[4]/div/div/div/div[4]/button').click()
    driver.refresh()           
    return driver



#Preparando a aquisição das páginas
    #Criando informação sobre quem está fazendo o scrap
headers = {
    'User-Agent': 'João Lima, from PUC-MG University',
    'From': 'jbetol@gmail.com'
}

#Ainda precisa Verificar com Try Except se driver está dispoível
try: #if os.path.exists('/media/joao/HDLinux/tools/'):
    driver = webdriver.Firefox('/media/joao/HDLinux/tools')
except IOError:
    sys.exit("Geckodriver not found!")
    
url_aliexpress = 'https://www.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=phone+display+lcd+for+screen&ltype=wholesale&SortType=default&page='

url_alibaba = 'https://www.alibaba.com/products/phone_display_lcd_screen/CID5904002.html?spm=a2700.galleryofferlist.0.0.126c1524tZVgH8&IndexArea=product_en&page='

#Vamos pegar 50 páginas de dados e jogar num array
paginas = []
qnt_paginas = 2
passou = False
alibaba = False  # Vamos começar scrapping por AliExpress.com
site_index = 2  # números de sites a ser scrapping: AliExpress e Alibaba
descricao = []
preco_min = []
preco_max = [] #_items = {} # Dicionário que vai guardar os dados das páginas in memory 
while site_index > 0  :
    site_index = site_index - 1
    for i in range(1,qnt_paginas):
        if not alibaba:
            url = url_aliexpress + str(i)
        else:
            url = url_alibaba + str(i)

        paginas.append(url)

    #loop para navegar em 30 páginas de aquisição de dados
    for pagina in paginas:
        driver.get(pagina)
        #driver.execute_script("arguments[0].setAttribute('lang', 'us')", elemento)#, 'class', 'vote-link up voted')

        if not alibaba and not passou:
            alteraIdioma(driver)
            passou = True

    #Verificar modo silencioso para o geckodriver
        
        req = driver.page_source

        soup = BeautifulSoup(req, "html.parser")

        #Aqui é para páginas estáticas. Não usa o webdriver
        #req = requests.get(url)
        #soup = BeautifulSoup(req.text, "html.parser")
        #Adquirindo a lista dos produtos que se encontra na classe indicada
        if alibaba:
            classes = soup.find('div', class_ = "organic-list app-organic-search__list")
            #Agora vamos buscar os itens da lista
            items =  classes.find_all(class_ = 'organic-gallery-offer-outter J-offer-wrapper')
        else:
            classes = soup.find('ul', class_ = "list-items")
            #Agora vamos buscar os itens da lista
            items =  classes.find_all('li', class_ = 'list-item')

        #Para cada item, pegar apenas descrição e o preco e guardar num dicionário
        for i,item in enumerate(items):
            if alibaba:
                descricao.append(item.find('h4').get('title'))
                preco = item.find('p', class_ = 'elements-offer-price-normal').get('title')
            else:
                descricao.append(item.find('a', class_ = 'item-title').get('title'))
                preco = item.find('span', class_ = 'price-current').text

            preco_min_max = re.findall(r'(\d+[\.,]\d{1,2})', preco)
            if len(preco_min_max) == 1:
                preco_min_max.insert(0,'0.00') 
            preco_min.append(preco_min_max[0])
            preco_max.append(preco_min_max[1])
            
    alibaba = True
    paginas.clear()

driver.quit()
#transformando o dicionário em DataFrame do Pandas
df = pd.DataFrame({
    'descricao': descricao,
    'preco_minimo': preco_min,
    'preco_maximo': preco_max
    })
    #"desc_items, columns=['Item','Descrição','Preço Mínimo', 'Preço Máximo'])
df.info()
df.head()
# displaying the DataFrame  
print('DataFrame:\n', df) 

# saving the DataFrame as a CSV file 
gfg_csv_data = df.to_csv('lista.csv', header = True) 




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   descricao     32 non-null     object
 1   preco_minimo  32 non-null     object
 2   preco_maximo  32 non-null     object
dtypes: object(3)
memory usage: 896.0+ bytes
DataFrame:
                                             descricao preco_minimo  \
0   UMIDIGI BISON IP68/IP69K Waterproof 128GB 6GB ...       169.99   
1   Sticker Adhesive-Tape Cellphone Touch-Screen M...         0.01   
2   Bubble-Remove Laminating-Machine Vacuum-Pump i...         0.00   
3   Glue Mobile-Phone-Repair-Machine Samsung Repai...         0.00   
4   Display For Xiaomi Redmi Go Lcd Display Touch ...        18.43   
5   Sensor Armor Lcd-Display Phone-Accessories Tou...         0.00   
6   6.59"Original Amoled For Asus ROG 3 ZS661KS LC...       164.70   
7   2/3/5/8mm Adhesive-Tape Housing-Tool Hardware ...         0.87   
8

In [1]:
import re
dici = {}
fonte = '$10.63-$16.82' # - US $43.77 - 44.27 $15.00-$ 39.81 US $18.49 - US325,30 US $12,70'
fonte_limpa = re.findall(r'(\d+[\.,]\d{1,2})', fonte) #re.sub(r'[aA-Zz$\-\s+]','',fonte)
#fonte_limpa = re.sub(r'[aA-Zz]','',fonte_limpa)
dici = fonte,fonte_limpa[0], fonte_limpa[1]
print(dici)

#print(re.findall(r'(\d+[\.,]\d{1,2})', fonte))


('$10.63-$16.82', '10.63', '16.82')


In [20]:
"""         if len(preco_min_max) > 1:
                desc_items['preco_min'] = preco_min_max[0]
                desc_items['preco_max'] = preco_min_max[1]
            else:
                desc_items['descricao'] = descricao
                desc_items['preco_min'] = preco_min_max[0]
                desc_items['preco_max'] = '0.00' 
"""


('Hot cellphone touch displays for iPhone 11, mobile phone lcd screens replacement LCD display for iPhone 11 screen',
 '8.88',
 '38.71')